## Concise: 1: Start and mesh
Author: yanjun zhang, Source from book "Abali - 2017 - Computational Reality" P119

In [1]:
from disc_f_core import *

# mesh-size, contact area coefficient
mesh_min, mesh_max = 3, 5
c_contact          = 1
# Each time step rotation angular, and acc during lag, 1 is full acc, 0 is no acc.
angular_r          = 224
v_vehicle, c_acc   = 160, 1
z1,z2,z3,z_all     = 20,33,30,8
pad_v_tag          = 32
# calling local functions to get all parameters
(dt, P, g, num_steps, h, radiation, v_angular, Ti, Tm, S_rub_circle, t, rho, c, k, t_brake, 
S_total,) = vehicle_initial (angular_r, v_vehicle, c_contact, c_acc)
print("1: Total braking tims is ", round(sum(dt), 2), "s")
print("2: Total numb steps is ", num_steps)



def solve_heat(Ti, u, num_steps, dt, x_co, y_co, angular_r, \
               t_brake, domain, S_rub_circle, fdim,\
               rho, c, v, radiation, k, h, f, Tm, g,\
               ds, xdmf, b_con, bc, plotter, warped ):
    import numpy as np
  
    T_array = [(0, [Ti for _ in range(len(u.x.array))])]
    total_degree = 0
    t = 0
    fraction_c = []
    for i in range(num_steps):
        
         t += dt[i]
        
         if i == 0: 
            u.x.array[:] = np.full(len(u.x.array), Ti)       
        
         if i ==0:
            x_co_new = x_co
            y_co_new = y_co
         else:
            pass     
   
         total_degree += angular_r  # Incrementing degree in each step  
             
         # Construct the message     
         end_time = time.time()
         elapsed_time = end_time - start_time
         elapsed_time1 = round(elapsed_time, 0)
         formatted_start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(start_time))
         if elapsed_time1 >= 60:
            min = elapsed_time1 / 60;       hours = min / 60
            progress_message = f"1: Progress: {round(100 * (t / t_brake), 1)}%. Use time: \
            {round(hours)} hours {round(min)} min. Start: {formatted_start_time }."
         else:
            progress_message = f"1: Progress: {round(100 * (t / t_brake), 1)}%. Use time: \
            {round(elapsed_time1)} s. Start: {formatted_start_time }."
         sys.stdout.write(f"\r{progress_message.ljust(80)}")  # 80 spaces to ensure full clearing
         sys.stdout.flush()

     

         F = ((rho * c) / dt[i] * inner(u, v) * dx
             + k * inner(grad(u), grad(v)) * dx
             + h * inner(u, v) * ds(b_con)
             + radiation * inner(u**4, v) * ds(b_con)
             - ( inner(f, v) * dx
                 + (rho * c) / dt[i] * inner(u_n, v) * dx  #!!!!!!!!!!!!   u_n need to double check
                 + h * Tm * v * ds(b_con)
                 + radiation * (Tm**4) * v * ds(b_con)) )
        
         g[i] = g[i]/ fraction_c1

         for j in list(range(1, 19)):
             #F += -k * dot(grad(u) * v, n_vector) * ds(10 * j) - inner(g[i], v) * ds(10 * j)
             F += ( - inner(g[i], v) * ds(10 * j) 
                    - h * inner( u, v) * ds(10 * j)  
                    - radiation * inner( (u**4 - Tm**4), v) * ds(10 * j) )    
     

         problem = NonlinearProblem(F, u, bcs=[bc])
         ## 7: Using petsc4py to create a linear solver
         solver_setup_solve(problem, u)
        
         u.x.scatter_forward()            
       
         # Update solution at previous time step (u_n)
         u_n.x.array[:] = u.x.array
         T_array.append((t, u.x.array.copy()))
         # Write solution to file
         xdmf.write_function(u, t)
         # Update plot
         #warped = grid.warp_by_scalar("uh", factor=0)
         plotter.update_coordinates(warped.points.copy(), render=False)
         plotter.update_scalars(u.x.array, render=False)
         plotter.write_frame()      

         print('Rub radius square is ', r_rub_new)

    plotter.close()
    xdmf.close()
    print()
    return(T_array, fraction_c, deformed_co,u_d1  )



## here use lots of abbreviation, details are in disc_f
domain, cell_markers, facet_markers, mesh_name, mesh_name1, mesh_name2 \
                       = mesh_brake_all(mesh_min,mesh_max,pad_v_tag)
V, T_init, u_n         = initial_u_n(domain, Ti)
fdim, bc, mesh_brake, all_e,xdmf, x_co, y_co, ds, b_con \
                       = mesh_setup(domain, V,mesh_name1,num_steps, \
                         angular_r, mesh_name2, c_contact,z_all,Tm, S_rub_circle)
# Initialize
problem,u,v,f,n_vector = variation_initial(V, T_init,domain, rho, c, b_con,\
                          radiation, h, k, xdmf,dt,ds,u_n, Tm,g,bc);
n,converged = solver_setup_solve(problem,u)

## Visualization of time dependent problem using pyvista
gif_name    = "T-s-{}-d-{}-{}-c-{}-e-{}.gif".format(num_steps, angular_r, mesh_name2, c_contact, all_e)
plotter, sargs, renderer, warped, viridis, grid = plot_gif(V,u,gif_name)
##solve
#num_steps= int(num_steps/10/6)
num_steps= int(10) # just do one step to check the deformation.


T_array     = solve_heat(Ti, u, num_steps, dt, x_co, y_co, angular_r, \
               t_brake, domain, S_rub_circle, fdim,\
               rho, c, v, radiation, k, h, f, Tm, g,\
               ds, xdmf, b_con, bc, plotter, warped )


csv_name    = "Result_T-s-{}-d-{}-{}-c-{}-e-{}.csv".format(num_steps, angular_r, mesh_name2, c_contact, all_e  )
# got the Temperature data
save_t_T(csv_name, T_array)

from IPython.display import display, Image
display(Image(gif_name))

DOLFINx version: 0.8.0
Simulation environment setup complete.
1: Total braking tims is  49.89 s
2: Total numb steps is  608
The file 'm-3-5.msh' exists, start creat now:

1: Progress: 0.1%. Use time:             11 s. Start: 2025-01-20 08:49:41.      

NameError: name 'fraction_c1' is not defined

# 2: New brake pad points, add boundary for rubbing elements

In [ ]:
## plot the temperature
mesh_n_pad = mesh_del_disc(mesh_name1, "new_pad.msh")
T_new_p, pad_node_coordinates  = T_pad_transfer1( mesh_name1, mesh_n_pad, u_n, mesh_brake, pad_v_tag )
domain_pad, cell_mark_pad, facet_mark_pad = gmshio.read_from_msh( mesh_n_pad , MPI.COMM_WORLD, 0, gdim=3 )
plot_T_pad( domain_pad, T_new_p).show()

# defin the pad domain
VT      = fem.functionspace(domain_pad, ("CG", 1))         #define the finite element function space
Delta_T = fem.Function(VT, name ="Temperature_variation")  # T_ is the test function, like v

plt.savefig('mesh_pad.png')

In [ ]:
T_new_p1 = []
for i in range(len(T_new_p)):
    T_new_p1 = T_new_p1 + [60]

In [ ]:



for i in range(len(T_new_p)): 
    Delta_T.x.array[i] = T_new_p[i]-T_new_p1[i]    #Delta_T is the nodes temperature. 

#######try to make domain only for brake pad.
E    = fem.Constant(domain_pad, 1.8e5)            # Elastic module
nu   = fem.Constant(domain_pad, 0.2)              # Poission ratio
gdim = domain_pad.geometry.dim

mu    = E / 2 / (1 + nu)                          # Shear modulus
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)          # Lame parameters
#alpha = fem.Constant(domain_pad, 1.5e-6)         # Thermal expansion coefficient
alpha = fem.Constant(domain_pad, 1.5e-5)   
f1    = fem.Constant(domain_pad, (0.0, 0.0, 0.0)) # O for external force

def eps(v):                                       # epsilon, strain, the deforamtion, dy/y 
    return ufl.sym(ufl.grad(v))
def sigma(v, Delta_T):                            # sigmathis is sigma
    return (lmbda * ufl.tr(eps(v)) - alpha * (3 * lmbda + 2 * mu) * Delta_T 
    ) * ufl.Identity(gdim)  + 2.0 * mu * eps(v)   # here braces is important, can not be in above line

Vu = fem.functionspace(domain_pad, ("CG", 1, (gdim,))) 
du = ufl.TrialFunction(Vu)
u_ = ufl.TestFunction(Vu)

Wint = ufl.inner(sigma(du, Delta_T), eps(u_)) * ufl.dx  # here du is unkown
aM   = ufl.lhs(Wint)                                    # Wint is long and lhs can help to distinguish unkown and know.
LM   = ufl.rhs(Wint) + ufl.inner(f1, u_) * ufl.dx       # knows parameters are in lhs

def up_side(x):
    #return np.isclose(x[2], (5))
    return np.isclose(x[2], (z1+z2+z3))

up_dofs_u = fem.locate_dofs_geometrical(Vu, up_side)    # lateral sides of domain
bcu       = [fem.dirichletbc(np.zeros((gdim,)), up_dofs_u, Vu)]  # displacement Vu is fixed in lateral sides

u_d       = fem.Function(Vu, name="Displacement")
problem   = fem.petsc.LinearProblem(aM, LM, u=u_d, bcs=bcu)
problem.solve()

scale_factor = 100
plot_s_pad   = plot_S_pad(Vu,u_d,scale_factor )
plot_s_pad.show()
plt.savefig('displacement_no_contact.png')

In [ ]:
scale_factor = 50000
plot_s_pad   = plot_S_pad(Vu,u_d,scale_factor )
plot_s_pad.show()
plt.savefig('displacement_no_contact_scale_50000.png')

## Penalty method

In [ ]:
np.set_printoptions(threshold=np.inf)
plt.plot(Delta_T.x.array)
print( Delta_T.x.array )

In [ ]:
def bottom_contact_nodes(x):
    return np.isclose(x[2], z1)
contact_dofs = fem.locate_dofs_geometrical(Vu, bottom_contact_nodes)
penalty_param = 400
# Create a function to store penalty forces in the same function space as displacement
penalty_forces = fem.Function(Vu)
def update_penalty_force(u_d, penalty_forces, z1, penalty_param):
    u_vals = u_d.x.array.reshape(-1, gdim)
    penalty_forces_vals = penalty_forces.x.array.reshape(-1, gdim)
    # Apply penalty force for nodes below z1
    for dof in contact_dofs:
        if u_vals[dof][2] < 0:  ## here should <0 because contact surface is minus once expend with no constrain.
            penalty_forces_vals[dof][2] = -penalty_param * ( u_vals[dof][2]) # if here is not minus, rubing element grew up
        else:
            penalty_forces_vals[dof][2] = 0.0  # No penalty force if above z1
    penalty_forces.x.array[:] = penalty_forces_vals.ravel()

u_d = fem.Function(Vu, name="Displacement")
problem = fem.petsc.LinearProblem(aM, LM, u=u_d, bcs=bcu)
problem.solve()


update_penalty_force(u_d, penalty_forces, z1, penalty_param)
u_d = fem.Function(Vu, name="Displacement")
LM_penalized = LM + ufl.inner(penalty_forces, u_) * ufl.dx
problem = fem.petsc.LinearProblem(aM, LM_penalized, u=u_d, bcs=bcu)
problem.solve()

scale_factor = 1
plot_s_pad = plot_S_pad(Vu, u_d, scale_factor)
plot_s_pad.show()
plt.savefig('displacement_contact.png')

## Get contact zone nodes

In [ ]:
x_co_zone   = 0.001
deformed_co, new_c = get_new_contact_nodes(x_co_zone, domain_pad, u_d,  Vu, z1, x_co, y_co )
x_co_new, y_co_new, r_rub_new, S_total_new,S_rub_circle_new = get_r_xco_yco (deformed_co, new_c )
fraction_c = []
fraction_c.append( (S_total_new)/(200) )
print('Total contact surface is: ', round(S_total_new, 2), " mm")
print("Contact friction is ", fraction_c)

In [ ]:
vtk = io.VTKFile(domain_pad.comm, "pad_deformation", "w")
vtk.write_function(u_d)
vtk.close()